In [1]:
import pandas as pd
import numpy as np
import random

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import rdkit
from rdkit.Chem import AllChem, rdFingerprintGenerator
from rdkit import Chem, DataStructs

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt

import catboost as cat

c:\Users\BSH\.virtualenvs\신약개발-mImSxWNo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RANDOM_SEED = 779

In [4]:
def feature_engineer():
  train_df = pd.read_csv("../data/train.csv")
  test_df = pd.read_csv("../data/test.csv")
  train_df = train_df.dropna()
  test_df["AlogP"] = np.where(pd.isna(test_df["AlogP"]), test_df["LogD"], test_df["AlogP"])
  
  train_df["mol"] = train_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  test_df["mol"] = test_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  
  train_df.drop(columns=["id", "SMILES"], inplace=True)
  test_df.drop(columns=["id", "SMILES"], inplace=True)
  
  fmgen = rdFingerprintGenerator.GetMorganGenerator()
  train_fps = train_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  train_fps = np.stack(train_fps)
  test_fps = test_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  test_fps = np.stack(test_fps)
  
  origin_train_features = train_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]].values
  origin_test_features = test_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]].values
  
  train_features = np.append(origin_train_features, train_fps, axis=1)
  test_features = np.append(origin_test_features, test_fps, axis=1)
  target = train_df["HLM"].values
  
  return train_features, target, test_features
  

In [5]:
train_x, train_y, test_x = feature_engineer()

In [6]:
def create_model(param):
  return cat.CatBoostRegressor(
    random_state=RANDOM_SEED,
    verbose=False,
    **param
  )

In [7]:
def train_model(param, X, y, X_test, trial=None):
    skf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

    val_scores = []
    y_tests = []
    models = []

    for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]

        model = create_model(param)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=50, verbose=500)

        y_hat_val = model.predict(X_val)
        score = mean_squared_error(y_val, y_hat_val, squared=True)
        val_scores.append(score)
        print(f'Fold: {idx+1}/5 score = {score:.5f}')

        y_tests.append(model.predict(X_test))
        models.append(model)

        if trial:
            trial.report(score, idx)

            if trial.should_prune():
                raise optuna.TrialPruned()

    return val_scores, y_tests, models

In [8]:
def objective_cat(trial):
    param = {
        "iterations": trial.suggest_int("iterations", 2000, 20000),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
    }

    val_scores, y_tests, models = train_model(param, train_x, train_y, test_x, trial)

    return sum(val_scores) / len(val_scores)

In [9]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='cat_tuning',
    pruner=HyperbandPruner(
        min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_cat, n_trials=50)

[I 2023-08-30 12:56:24,848] A new study created in memory with name: cat_tuning


0:	learn: 35.1256631	test: 35.1256631	test1: 35.0656409	best: 35.0656409 (0)	total: 206ms	remaining: 16m 50s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.52932247
bestIteration = 120

Shrink model to first 121 iterations.
Fold: 1/5 score = 994.09818
0:	learn: 34.7333382	test: 34.7333382	test1: 35.7462437	best: 35.7462437 (0)	total: 61ms	remaining: 4m 58s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 33.13820042
bestIteration = 52

Shrink model to first 53 iterations.
Fold: 2/5 score = 1098.14034
0:	learn: 34.9951707	test: 34.9951707	test1: 35.4233129	best: 35.4233129 (0)	total: 59.4ms	remaining: 4m 50s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.38666494
bestIteration = 111

Shrink model to first 112 iterations.
Fold: 3/5 score = 1048.89607
0:	learn: 35.0011311	test: 35.0011311	test1: 34.9505235	best: 34.9505235 (0)	total: 60.5ms	remaining: 4m 56s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.08

[I 2023-08-30 12:57:18,143] Trial 0 finished with value: 1040.1986782000695 and parameters: {'iterations': 4895, 'learning_rate': 0.21705661461621598, 'depth': 10, 'subsample': 0.3823213176469117, 'min_data_in_leaf': 86, 'colsample_bylevel': 0.5640302079822391}. Best is trial 0 with value: 1040.1986782000695.


Fold: 5/5 score = 1030.28195
0:	learn: 36.0642080	test: 36.0642080	test1: 35.9686005	best: 35.9686005 (0)	total: 3.39ms	remaining: 1m 3s
500:	learn: 31.6902155	test: 31.6902155	test1: 32.2059868	best: 32.2053546 (499)	total: 1.72s	remaining: 1m 2s
1000:	learn: 30.1188001	test: 30.1188001	test1: 31.7940560	best: 31.7940560 (1000)	total: 3.46s	remaining: 1m
1500:	learn: 28.8679563	test: 28.8679563	test1: 31.6089663	best: 31.6089663 (1500)	total: 5.17s	remaining: 58.9s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.49736591
bestIteration = 1942

Shrink model to first 1943 iterations.
Fold: 1/5 score = 992.08407
0:	learn: 35.8535536	test: 35.8535536	test1: 36.5464836	best: 36.5464836 (0)	total: 3.37ms	remaining: 1m 2s
500:	learn: 31.3360227	test: 31.3360227	test1: 33.5386237	best: 33.5386237 (500)	total: 1.71s	remaining: 1m 1s
1000:	learn: 29.7605877	test: 29.7605877	test1: 33.1713694	best: 33.1707524 (999)	total: 3.42s	remaining: 1m
Stopped by overfitting detector  (

[I 2023-08-30 12:57:58,364] Trial 1 finished with value: 1026.794869466957 and parameters: {'iterations': 18579, 'learning_rate': 0.013470177913990702, 'depth': 3, 'subsample': 0.6823721081214738, 'min_data_in_leaf': 43, 'colsample_bylevel': 0.2225891933670322}. Best is trial 1 with value: 1026.794869466957.


...

Fold: 5/5 score = 992.67602
0:	learn: 36.0584286	test: 36.0584286	test1: 35.9518486	best: 35.9518486 (0)	total: 4.21ms	remaining: 1m 21s
500:	learn: 27.3497667	test: 27.3497667	test1: 31.5332177	best: 31.4960628 (482)	total: 2.2s	remaining: 1m 23s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.4960628
bestIteration = 482

Shrink model to first 483 iterations.
Fold: 1/5 score = 992.00198
0:	learn: 35.7073768	test: 35.7073768	test1: 36.4196375	best: 36.4196375 (0)	total: 4.25ms	remaining: 1m 22s
500:	learn: 27.0860400	test: 27.0860400	test1: 32.9631776	best: 32.9587566 (499)	total: 2.19s	remaining: 1m 22s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.86424783
bestIteration = 637

Shrink model to first 638 iterations.


[I 2023-08-30 15:08:51,442] Trial 48 pruned. 


Fold: 2/5 score = 1080.05880
0:	learn: 35.7528006	test: 35.7528006	test1: 35.6713248	best: 35.6713248 (0)	total: 15.3ms	remaining: 4m 38s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.34670709
bestIteration = 268

Shrink model to first 269 iterations.
Fold: 1/5 score = 982.61606
0:	learn: 35.5244397	test: 35.5244397	test1: 36.3190780	best: 36.3190780 (0)	total: 14.4ms	remaining: 4m 22s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.76323945
bestIteration = 236

Shrink model to first 237 iterations.
Fold: 2/5 score = 1073.42987
0:	learn: 35.9460887	test: 35.9460887	test1: 36.3097924	best: 36.3097924 (0)	total: 4.29ms	remaining: 1m 17s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.09468631
bestIteration = 253

Shrink model to first 254 iterations.
Fold: 3/5 score = 1030.06890
0:	learn: 36.0078141	test: 36.0078141	test1: 35.9099493	best: 35.9099493 (0)	total: 15ms	remaining: 4m 32s
Stopped by overfitting detector  (50 ite

[I 2023-08-30 15:09:32,115] Trial 49 finished with value: 1005.362794378472 and parameters: {'iterations': 18180, 'learning_rate': 0.07211548346204227, 'depth': 8, 'subsample': 0.950474154829895, 'min_data_in_leaf': 66, 'colsample_bylevel': 0.10327285801962122}. Best is trial 18 with value: 993.6935440799392.


Fold: 5/5 score = 981.06552


In [10]:
trial = study.best_trial

In [11]:
print(trial.params)

{'iterations': 16968, 'learning_rate': 0.04295424452500349, 'depth': 9, 'subsample': 0.782000404237273, 'min_data_in_leaf': 100, 'colsample_bylevel': 0.6036379369859356}


In [12]:
best_param = {
  'iterations': 16968,
  'learning_rate': 0.04295424452500349,
  'depth': 9,
  'subsample': 0.782000404237273,
  'min_data_in_leaf': 100,
  'colsample_bylevel': 0.6036379369859356
}

val_scores, y_tests, models = train_model(best_param, train_x, train_y, test_x)

0:	learn: 35.8926360	test: 35.8926360	test1: 35.8079858	best: 35.8079858 (0)	total: 38.3ms	remaining: 10m 50s
500:	learn: 18.3062052	test: 18.3062052	test1: 30.8213061	best: 30.8206654 (498)	total: 21.2s	remaining: 11m 38s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.78558394
bestIteration = 569

Shrink model to first 570 iterations.
Fold: 1/5 score = 947.75219
0:	learn: 35.7331213	test: 35.7331213	test1: 36.4339028	best: 36.4339028 (0)	total: 33ms	remaining: 9m 19s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.64491535
bestIteration = 292

Shrink model to first 293 iterations.
Fold: 2/5 score = 1065.69051
0:	learn: 35.7917876	test: 35.7917876	test1: 36.1844146	best: 36.1844146 (0)	total: 36ms	remaining: 10m 11s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.95469403
bestIteration = 392

Shrink model to first 393 iterations.
Fold: 3/5 score = 1021.10248
0:	learn: 35.8661470	test: 35.8661470	test1: 35.7539930	best: 35.

In [13]:
submit = pd.read_csv("./catboost_optuna.csv")
submit["HLM"] = y_tests[-1]
submit.to_csv("catboost_optuna.csv", index=False)